In [1]:
from sqlalchemy import *
from sqlalchemy.schema import *
# from sqlalchemy import create_engine, inspect, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql import text
import contextlib 
import pandas as pd
import unicodedata
import json
# # Imports the methods needed to abstract classes into tables
# from sqlalchemy.ext.declarative import declarative_base

# # Allow us to declare column types
# from sqlalchemy import Column, Integer, String, Float 

In [11]:

rds_connection_string = "postgres:PostgreSQL@localhost:5432/trfmk_scrape_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
metadata = MetaData()
engine.execute(DropSchema('public', cascade = True))
engine.execute('CREATE SCHEMA IF NOT EXISTS public;')
    
# metadata.create_all(engine)
# inspector = inspect(engine)
# Base = declarative_base()


In [12]:
leagues_df = pd.read_csv("data/leagues_data.csv")
teams_df = pd.read_csv("data/teams_trmk.csv")
players_df = pd.read_csv("data/players_trmk1.csv")

In [13]:

players_df.to_sql(name = 'players',index_label= None, index= False, con=engine, if_exists='replace')
teams_df.to_sql(name = 'teams', index_label= None, index= False, con=engine, if_exists='replace')
leagues_df.to_sql(name = 'leagues', index_label= None, index= False, con=engine, if_exists='replace')

with engine.connect() as con:
    con.execute('ALTER TABLE leagues ADD CONSTRAINT pk_Leagues PRIMARY KEY ("league_ID")')
    con.execute('ALTER TABLE teams ADD CONSTRAINT pk_Teams PRIMARY KEY ("team_ID")')
    con.execute('ALTER TABLE players ADD CONSTRAINT pk_Players PRIMARY KEY ("players_ID")')
    con.execute('ALTER TABLE Teams ADD CONSTRAINT fk_Teams_league_ID FOREIGN KEY("league_ID") REFERENCES Leagues ("league_ID");')
    con.execute('ALTER TABLE Players ADD CONSTRAINT fk_Players_team_ID FOREIGN KEY("team_ID") REFERENCES Teams ("team_ID");')


# leagues_df.to_sql(name = 'leagues', con=engine, if_exists='replace' )
# teams_df.to_sql(name = 'teams', con=engine, if_exists='replace')
# players_df.to_sql(name = 'players', con=engine, if_exists='replace')



In [5]:
# engine.table_names()
# leagues.columns.league_id

In [6]:
# print(inspector.get_columns('leagues'))

In [7]:
# pd.read_sql_query('SELECT * FROM  leagues', con=engine)

In [14]:
pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING("team_ID")) USING ("league_ID")', con=engine)

,league_ID,country,tier,league_name,league_link,team_ID,club,squad,foreigners,avg_market_value_m,...,field_position,age,nat,height,foot,dt_joined,prev_team,contract_expires,market_value,player_page
0,1,BRAZIL,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,33,3,4.17,...,GLK,34,Brazil,1.87,left,7/16/2017,Valencia CF,12/31/2020,3.16,https://www.transfermarkt.com/diego-alves/prof...
1,1,BRAZIL,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,33,3,4.17,...,GLK,21,Brazil,1.96,both,8/31/2019,Clube de Regatas do Flamengo U20,9/30/2023,0.51,https://www.transfermarkt.com/hugo-souza/profi...
2,1,BRAZIL,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,33,3,4.17,...,GLK,28,Brazil,1.94,right,1/1/2013,Clube de Regatas do Flamengo U20,4/30/2022,0.45,https://www.transfermarkt.com/cesar/profil/spi...
3,1,BRAZIL,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,33,3,4.17,...,GLK,22,Brazil,1.88,right,1/1/2017,Clube de Regatas do Flamengo U20,12/31/2022,0.06,https://www.transfermarkt.com/gabriel-batista/...
4,1,BRAZIL,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,33,3,4.17,...,DEF,26,Brazil,1.83,right,1/1/2019,Sao Paulo Futebol Clube,12/31/2023,5.42,https://www.transfermarkt.com/rodrigo-caio/pro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2761,4,UNITED_STATES,2,USL Championship,https://www.transfermarkt.com/usl-pro/startsei...,101,Atlanta United 2,15,6,0.06,...,ATT,22,United States,1.70,left,1/14/2020,Atlanta United FC,11/30/2020,0.00,https://www.transfermarkt.com/phillip-goodrum/...
2762,4,UNITED_STATES,2,USL Championship,https://www.transfermarkt.com/usl-pro/startsei...,101,Atlanta United 2,15,6,0.06,...,ATT,17,United States,1.74,-,3/7/2020,Atlanta United Academy,None,0.00,https://www.transfermarkt.com/tyler-wolff/prof...
2763,4,UNITED_STATES,2,USL Championship,https://www.transfermarkt.com/usl-pro/startsei...,101,Atlanta United 2,15,6,0.06,...,ATT,17,United States,1.68,-,3/7/2020,Atlanta United Academy,None,0.00,https://www.transfermarkt.com/coleman-gannon/p...
2764,4,UNITED_STATES,2,USL Championship,https://www.transfermarkt.com/usl-pro/startsei...,101,Atlanta United 2,15,6,0.06,...,ATT,18,United States,1.88,-,12/20/2018,Atlanta United Academy,11/30/2020,0.17,https://www.transfermarkt.com/jackson-conway/p...


In [10]:
pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING("team_ID")) USING ("league_ID")', con=engine)\
.to_json(r'dashboard/static/data/data.json', orient='records')

In [ ]:
# metadata = MetaData()
# engine.execute(DropSchema('public', cascade = True))
# engine.execute(CreateSchema('public'))

# leagues = Table('leagues', metadata,
#     Column('league_id', Integer, primary_key=True),
#     Column('country', String(255), nullable=False),
#     Column('tier', Integer, nullable=False),    
#     Column('league_name', String(255), nullable=False),
#     Column('league_link', String(255), nullable=False),
#     schema='public'
# )

# teams = Table('teams', metadata,
#     Column('team_id', Integer, primary_key=True),
#     Column('league_id', Integer,ForeignKey("leagues.league_id")),
#     Column('club', String(255), nullable=False),
#     Column('squad', Integer, nullable=False),
#     Column('foreigners', Integer, nullable=False),
#     Column('avg_market_value_m', Float, nullable=False),
#     Column('total_MV_m', Float, nullable=False),
#     Column('Logo_img', String(255), nullable=False),
#     Column('link_page', String(255), nullable=False),
#     schema='public'
# );

# players = Table( 'players', metadata,
#     Column('players_id', Integer, primary_key=True),
#     Column('team_id', Integer,ForeignKey("teams.team_id")),
#     Column('name', String(255), nullable=False),
#     Column('position', String(255), nullable=False),
#     Column('field_position', String(255), nullable=False),
#     Column('Age', Integer, nullable=False),
#     Column('Nat', String(255), nullable=False),
#     Column('Height', Float, nullable=False),
#     Column('foot', String(255), nullable=False),
#     Column('dt_joined', Date, nullable=False),
#     Column('prev_team', String(255), nullable=False),
#     Column('contract_expires', Date, nullable=False),
#     Column('market_value', Float, nullable=False),
#     Column('player_page', String(255), nullable=False),
#     schema='public'
# );

# metadata.create_all(engine)

In [ ]:
# Leagues
# -
# league_ID Interger PK
# country  String(255)
# tier    Interger
# league_name String(255)
# league_link


# Teams
# -
# team_ID Integer PK
# league_ID Integer FK >- Leagues.league_ID
# club String(255)
# squad Integer
# foreigners Integer
# avg_market_value_m Float
# total_MV_m Float
# Logo_img String(255)
# link_page String(255)

# Players
# -
# players_ID            Integer PK
# team_ID               Integer FK >- Teams.team_ID
# name                 String(255)
# position             String(255)
# Age                   Integer
# Nat                  String(255)
# Height              Float
# foot                 String(255)
# dt_joined            String(255)
# prev_team            String(255)
# contract_expires     String(255)
# market_value        Float
# player_page          String(255)

# Player_status
# -
# players_ID  Integer FK >- Players.players_ID
# league_ID   Interger FK >- Leagues.league_ID
# appearances Interger 
# Goals   Interger 
# assists   Interger
# clean_sheats   Interger
# goals_conceded   Interger
# minutes_played  Interger 